In [1]:
import os
import requests
import pandas as pd
import io
import csv

In [3]:
# filename = 'uscensus/CouncilDist 2 and 3-2024.csv'
filename = '/opt/data/GIS-Census/florida/gisdata2/CouncilDist 2 and 3-2024.csv'
dfDistrict = pd.read_csv(filename, sep=',', engine='python')

In [4]:
# import os
# # Get the current working directory
# current_directory = os.getcwd()
# 
# print(current_directory)
# dfDistrict = dfDistrict.dropna(how='any')# 

In [5]:
dfDistrict.shape

(98241, 51)

In [6]:
df_addresses = dfDistrict.filter(items=['VoterID', 'Residence_Address', 'City_Name', 'Zip_Code'])
df_addresses['State'] = 'Fl'

In [7]:
len(df_addresses)

98241

In [10]:
df_short = df_addresses.head(100)
df_short

,VoterID,Residence_Address,City_Name,Zip_Code,State
0,128944025,11302 Soforenko Dr,Jacksonville,32218,Fl
1,126258777,10843 Happy Vale Rd,Jacksonville,32246,Fl
2,115266835,10843 Happy Vale Rd,Jacksonville,32246,Fl
3,129227985,10843 Happy Vale Rd,Jacksonville,32246,Fl
4,109927046,10871 Java Dr,Jacksonville,32246,Fl
...,...,...,...,...,...
95,125793919,10727 Majuro Dr,Jacksonville,32246,Fl
96,127846568,1250 Brookwood Forest Blvd 6104,Jacksonville,32225,Fl
97,129962437,12271 Bonnet Ct,Jacksonville,32226,Fl
98,129064075,3997 Hammock Bluff Cir,Jacksonville,32226,Fl


In [11]:
url = 'https://geocoding.geo.census.gov/geocoder/geographies/addressbatch'
payload = {'benchmark':'Public_AR_Current','vintage':'Current_Current'}

In [13]:
# batch_size = 9900
batch_size = 10
print(f"Batch size: {batch_size}")
dfCensusResults = pd.DataFrame()

Batch size: 10


In [15]:
# for i in range(0, len(df_addresses), batch_size):
for i in range(0, len(df_short), batch_size):
    df_address_chunk = df_addresses[i:i + batch_size] 
    # address_chunk_name = f"uscensus/district_addresses-{i}.csv"
    address_chunk_name = f"district_addresses-{i}.csv"
    print(f"{i}) Process batch: ({address_chunk_name}")
    if(os.path.exists(address_chunk_name)):
        # df_address_chunk = pd.read_csv(address_chunk_name, sep=',', header=None)
        print(f"{i}) Source Chunk File exists:{address_chunk_name}")
    else:
        print(f"{i}) Write address chunk:", address_chunk_name)
        df_address_chunk.to_csv(address_chunk_name, index=False, header=False)

    response_chunk_name = f"district_responses-{i}.csv"
    # response_chunk_name = f"uscensus/district_responses-{i}.csv"
    if(os.path.exists(response_chunk_name)):
        # df_response_chunk = pd.read_csv(response_chunk_name, sep=',', header=None)
        print(f"{i}) RESPONSE chunk File exists:{address_chunk_name}")
    else:
        print(f"{i}) Prepare to write response chunk:{response_chunk_name} from address file: {address_chunk_name}")
        files = {'addressFile': (filename, open(address_chunk_name, 'rb'), 'text/csv')}
        s = requests.post(url, files=files, data=payload, timeout=None)
        print("\t\tGot reponse from Census API")
        response_chunk_file = open(response_chunk_name, 'wb')
        response_chunk_file.write(s.content)
        response_chunk_file.close()
        # raw_file = open(f"raw_file-{i}.csv", 'rb+')
        # df_response_chunk = pd.read_csv(raw_file, sep=',', header=None)
    
    # dfCensusResults = dfCensusResults.append(df_response_chunk, ignore_index=True)
    # out_results = f"district-census-{i}.csv"
    # dfCensusResults.to_csv(out_name, index=False)
    

0) Process batch: (district_addresses-0.csv
0) Write address chunk: district_addresses-0.csv
0) Prepare to write response chunk:district_responses-0.csv from address file: district_addresses-0.csv
		Got reponse from Census API
10) Process batch: (district_addresses-10.csv
10) Write address chunk: district_addresses-10.csv
10) Prepare to write response chunk:district_responses-10.csv from address file: district_addresses-10.csv
		Got reponse from Census API
20) Process batch: (district_addresses-20.csv
20) Write address chunk: district_addresses-20.csv
20) Prepare to write response chunk:district_responses-20.csv from address file: district_addresses-20.csv
		Got reponse from Census API
30) Process batch: (district_addresses-30.csv
30) Write address chunk: district_addresses-30.csv
30) Prepare to write response chunk:district_responses-30.csv from address file: district_addresses-30.csv
		Got reponse from Census API
40) Process batch: (district_addresses-40.csv
40) Write address chunk: 

In [ ]:
dfCensusResults.head()

In [ ]:
dfCensusResults.dtypes

In [ ]:
dfCensusResults.columns = ['id', 'ADDRESS_IN', 'MATCH_INDICATOR', 'MATCH_TYPE', 'ADDRESS_OUT', 'LONG_LAT', 'TIGER_EDGE', 'STREET_SIDE', 'FIPS_STATE', 'FIPS_COUNTY', 'CENSUS_TRACT', 'CENSUS_BLOCK']

In [ ]:
# dfCensus1 = dfCensusResults.loc[dfCensusResults['ADDRESS_IN'].str.contains('95 Jardin')]
# dfCensus1.head()

In [ ]:
hot_dems_file = 'hot-dems.csv'
dfHotDems = pd.read_csv(hot_dems_file, sep=',', quoting=csv.QUOTE_ALL)
dfHotDems.head(10)

In [ ]:
dfHotDems.dtypes

In [ ]:
# dfHotDems1 = dfHotDems.loc[dfHotDems['Address'].str.contains('95 Jardin')]
# dfHotDems1.head(10)

In [ ]:
dfCombined = dfCensusResults.join(dfHotDems, on='id', how='left', lsuffix='_left', rsuffix='_right')

In [ ]:
dfCombined.dtypes

In [ ]:
dfCombined.head()

In [ ]:
# dfCombined1 = dfCensusResults.join(dfHotDems, on='id')

In [ ]:
# dfCombined1.head()

In [ ]:
# dfCombined2 = dfCensusResults.merge(dfHotDems)

In [ ]:
# dfCombined2.head()

In [ ]:
dfCombined.to_csv('combined.csv')